In [ ]:
from keras import *
from keras.utils import Sequence
from keras.models import Model
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import os
import matplotlib.pyplot as plt
from vis.utils import utils
import tensorflow as tf
import random as rn
import cv2
import keras.backend as K
rn.seed(666)
tf.set_random_seed(666)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


def compare_two_layer(layer1, layer2, m):
    idx1, idx2 = utils.find_layer_idx(m, layer1), utils.find_layer_idx(m, layer2)
    model1 = Model(inputs=m.inputs, outputs=m.layers[idx1].output)
    model2 = Model(inputs=m.inputs, outputs=m.layers[idx2].output)
    maps1 = model1.predict([X1[0].reshape(1, 50, 72, 106, 1), X2[0].reshape(1, 50, 13, 13, 1),
                            X3[0].reshape(1, 13, 13, 1), X4[0].reshape(1, 13, 13, 1)])
    maps2 = model2.predict([X1[0].reshape(1, 50, 72, 106, 1), X2[0].reshape(1, 50, 13, 13, 1),
                            X3[0].reshape(1, 13, 13, 1), X4[0].reshape(1, 13, 13, 1)])
    width = maps1.shape[3] * 10 if maps1.shape[3] > 10 else maps1.shape[3] * 20
    height = maps1.shape[2] * 10 if maps1.shape[3] > 10 else maps1.shape[2] * 20
    print(maps1.shape)
    print(np.array_equal(maps1, maps2))
    # video = cv2.VideoWriter("fmaps/L1.mp4", -1, 3, (width, height))
    video = cv2.VideoWriter("fmaps/compare.mp4", -1, 5, (width*2, height))
    for _ in range(maps1.shape[1]):
        map1 = maps1[0, _, :, :, 1]
        map2 = maps2[0, _, :, :, 1]
        im = np.hstack((map1, map2))
        im = cv2.resize(im, (width*2, height), interpolation=cv2.INTER_NEAREST).astype(np.uint8)
        video.write(im*30 + 80)
        # cv2.imshow('final', np.where((255 - im) < 80, 255, im+80))
        # cv2.imshow('final', Contrast_and_Brightness(10, 50, im))
        # cv2.imshow('final', im*50+70)
        # k = cv2.waitKey(60) & 0xff
        # if k == 27:
        #     break
    cv2.destroyAllWindows()
    video.release()


def plot_all_maps(layer):
    print(layer)
    idx = utils.find_layer_idx(model, layer)
    activation_model = Model(inputs=model.inputs, outputs=model.layers[idx].output)
    feature_maps = activation_model.predict([X1[0].reshape(1, 50, 72, 106, 1), X2[0].reshape(1, 50, 13, 13, 1),
                                             X3[0].reshape(1, 13, 13, 1), X4[0].reshape(1, 13, 13, 1)])
    feature_maps = feature_maps.reshape((1, 50, 3, 5, 32))
    print(feature_maps.shape)
    width = feature_maps.shape[3] * 200
    height = feature_maps.shape[2] * 400
    # print(height)
    # video = cv2.VideoWriter("fmaps/L1.mp4", -1, 3, (width, height))
    video = cv2.VideoWriter("fmaps/" + layer + "_all_maps.mp4", -1, 3, (width, height))
    id = 0
    for _ in range(feature_maps.shape[1]):
        # specify subplot and turn of axis
        # ax = plt.subplot(h, w, ix)
        # ax.set_xticks([])
        # ax.set_yticks([])
        # plot filter channel in grayscale
        map = feature_maps[0, _, :, :, 0]
        for y in range(1, 4, 1):
            map = np.concatenate((map, feature_maps[0, _, :, :, y]), axis=1)
        for x in range(1, 8, 1):
            tmp = feature_maps[0, _, :, :, x*4]
            for y in range(1, 4, 1):
                tmp = np.concatenate((tmp, feature_maps[0, _, :, :, y+x*y]), axis=1)
            map = np.concatenate((map, tmp), axis=0)
        map = cv2.resize(map, (width, height), interpolation=cv2.INTER_NEAREST)
        map = (map*5000 + 50).astype(np.uint8)
        for i in range(1, 4):
            cv2.line(map, ((width*i)//4, 0), ((width*i//4), height), (255, 255, 255), 2)
        for i in range(1, 8):
            cv2.line(map, (0, (height*i)//8), (width, (height*i)//8), (255, 255, 255), 2)
        # cv2.imwrite("fmaps/dense/f%d.png" % id, map)
        # id += 1
        video.write(map)
        # cv2.imshow('final', np.where((255 - im) < 80, 255, im+80))
        # cv2.imshow('final', Contrast_and_Brightness(10, 50, im))
        # cv2.imshow('final', map)
        # k = cv2.waitKey(60) & 0xff
        # if k == 27:
        #     break
    cv2.destroyAllWindows()
    video.release()


def compare_all_maps(layer1, layer2, m):
    idx1, idx2 = utils.find_layer_idx(m, layer1), utils.find_layer_idx(m, layer2)
    model1 = Model(inputs=m.inputs, outputs=m.layers[idx1].output)
    model2 = Model(inputs=m.inputs, outputs=m.layers[idx2].output)
    maps1 = model1.predict([X1[0].reshape(1, 50, 72, 106, 1), X2[0].reshape(1, 50, 13, 13, 1),
                            X3[0].reshape(1, 13, 13, 1), X4[0].reshape(1, 13, 13, 1)])
    maps2 = model2.predict([X1[0].reshape(1, 50, 72, 106, 1), X2[0].reshape(1, 50, 13, 13, 1),
                            X3[0].reshape(1, 13, 13, 1), X4[0].reshape(1, 13, 13, 1)])
    id = 0
    for _ in range(maps1.shape[1]):
        # specify subplot and turn of axis
        # ax = plt.subplot(h, w, ix)
        # ax.set_xticks([])
        # ax.set_yticks([])
        # plot filter channel in grayscale
        map1 = maps1[0, _, :, :, 0]
        for y in range(1, 4, 1):
            map1 = np.concatenate((map1, maps1[0, _, :, :, y]), axis=1)
        for x in range(1, 8, 1):
            tmp = maps1[0, _, :, :, x*4]
            for y in range(1, 4, 1):
                tmp = np.concatenate((tmp, maps1[0, _, :, :, y+x*y]), axis=1)
            map1 = np.concatenate((map1, tmp), axis=0)

        map2 = maps2[0, _, :, :, 0]
        for y in range(1, 4, 1):
            map2 = np.concatenate((map2, maps2[0, _, :, :, y]), axis=1)
        for x in range(1, 8, 1):
            tmp = maps2[0, _, :, :, x * 4]
            for y in range(1, 4, 1):
                tmp = np.concatenate((tmp, maps2[0, _, :, :, y + x * y]), axis=1)
            map2 = np.concatenate((map2, tmp), axis=0)
        print("f%d:" % id, np.array_equal(map1, map2))
        id += 1


def plot_layers_map():
    for layer in layers:
        print(layer)
        idx = utils.find_layer_idx(model, layer)
        activation_model = Model(inputs=model.inputs, outputs=model.layers[idx].output)
        feature_maps = activation_model.predict([X1[0].reshape(1, 50, 72, 106, 1), X2[0].reshape(1, 50, 13, 13, 1),
                                                 X3[0].reshape(1, 13, 13, 1), X4[0].reshape(1, 13, 13, 1)])
        feature_maps = feature_maps.reshape((1, 50, 3, 5, 32))
        print(feature_maps.shape)
        width = feature_maps.shape[3] * 10 if feature_maps.shape[3] > 10 else feature_maps.shape[3] * 60
        height = feature_maps.shape[2] * 10 if feature_maps.shape[3] > 10 else feature_maps.shape[2] * 60
        # print(height)
        # video = cv2.VideoWriter("fmaps/L1.mp4", -1, 3, (width, height))
        video = cv2.VideoWriter("fmaps/" + layer + "_2.mp4", -1, 3, (width, height))
        id = 0
        for _ in range(feature_maps.shape[1]):
            # specify subplot and turn of axis
            # ax = plt.subplot(h, w, ix)
            # ax.set_xticks([])
            # ax.set_yticks([])
            # plot filter channel in grayscale
            map = np.mean(feature_maps[0, _, :, :, :], axis=2)
            im = cv2.resize(map, (width, height), interpolation=cv2.INTER_NEAREST)
            # cv2.imwrite("fmaps/dense/f%d.png" % id, (im*100 + 50).astype(np.uint8))
            # id += 1
            video.write((im*5000 + 60).astype(np.uint8))
            # cv2.imshow('final', np.where((255 - im) < 80, 255, im+80))
            # cv2.imshow('final', Contrast_and_Brightness(10, 50, im))
            # cv2.imshow('final', im)
            # k = cv2.waitKey(60) & 0xff
            # if k == 27:
            #     break
        cv2.destroyAllWindows()
        video.release()


def output_heatmap(model, last_conv_layer, img):
    """Get the heatmap for image.

    Args:
           model: keras model.
           last_conv_layer: name of last conv layer in the model.
           img: processed input image.

    Returns:
           heatmap: heatmap.
    """
    # predict the image class
    preds = model.predict(img)
    # find the class index
    index = np.argmax(preds[0])
    # This is the entry in the prediction vector
    target_output = model.output[:, index]

    # get the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer)

    # compute the gradient of the output feature map with this target class
    grads = K.gradients(target_output, last_conv_layer.output)[0]

    # mean the gradient over a specific feature map channel
    pooled_grads = K.mean(grads, axis=(0, 1, 2, 3))

    # this function returns the output of last_conv_layer and grads
    # given the input picture
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([img])

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the target class

    for i in range(conv_layer_output_value.shape[-1]):
        conv_layer_output_value[:, :, :, i] *= pooled_grads_value[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    # heatmap = np.maximum(heatmap, 0)
    # heatmap /= np.max(heatmap)
    c = 0
    for i in heatmap[10:21]:
        plt.axis('off')
        plt.tight_layout()
        # plt.imshow(i, clim=(0.0, 0.2))
        plt.imshow(i)
        plt.set_cmap('nipy_spectral')
        plt.savefig('fmaps/heatmap/f%d.png' % c, bbox_inches='tight', transparent="True", pad_inches=0)
        c += 1


def Contrast_and_Brightness(alpha, beta, img):
    blank = np.zeros(img.shape, img.dtype)
    # dst = alpha * img + beta * blank
    dst = cv2.addWeighted(img, alpha, blank, 1-alpha, beta)
    return dst


vds = ['nfbVD-N5-S-21.063-M-PS', 'nfbVD-N5-S-23.910-M-PS', 'nfbVD-N5-N-25.310-M-PS']
# vds = ['nfbVD-N5-S-23.910-M-PS', 'nfbVD-N5-N-25.310-M-PS']

cctvs = ['nfbCCTV-N5-N-16.915-M', 'nfbCCTV-N5-N-18.308-M', 'nfbCCTV-N5-N-22.514-M', 'nfbCCTV-N5-N-25.309-M',
         'nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M']
# cctvs = ['nfbCCTV-N5-S-23.896-M', 'nfbCCTV-N5-N-25.309-M']
test_day1 = datetime.strptime('20190606 1800', "%Y%m%d %H%M")
test_day1_str = test_day1.strftime("%Y%m%d %H%M")

batch_size = 32
frame_num = 50

model = models.load_model('model.h5')
cctv_Id = 'nfbCCTV-N5-N-16.915-M'
vd_Id = 'nfbVD-N5-S-21.063-M-PS'
print(cctv_Id)
vdDir = 'D:/vd/' + vd_Id + '/vd_speed/'
cctvDir = 'E:/wcs/cctv/' + cctv_Id + "/"
filename = vdDir + 'vdspd_20190321_20190421_nonzero.csv'
total_frames = cctvDir + "frame_num.txt"
# vd_df = pd.read_csv(filename, infer_datetime_format=True, index_col=0)
# vd_df.index = pd.to_datetime(vd_df.index)
# vd_df = vd_df.shift(-1)
# vd_df.loc[vd_df.index[len(vd_df)-1]] = 90
# frame_df = pd.read_csv(total_frames, infer_datetime_format=True, index_col=0, header=None)
# frame_df.index = pd.to_datetime(frame_df.index)

X1 = []
X2 = []
X3 = []
X4 = []

x_1 = list(np.load(cctvDir + test_day1.strftime("%Y%m%d") + "_" + test_day1.strftime("%H%M") + '.npy')[:frame_num])
x_2 = list(np.load(cctvDir + test_day1.strftime("%Y%m%d") + "_" + test_day1.strftime("%H%M") + '_timestamp.npy')[:frame_num])
X3.append(x_2[len(x_2) - 1])
X4.append(x_2[0])
while len(x_1) < frame_num:
    x_1.append(np.zeros_like(x_1[0]))
    x_2.append(np.zeros_like(x_2[0]))
X1.append(x_1[:frame_num])
X2.append(x_2[:frame_num])
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
X4 = np.array(X4)
print("Test length:", len(X1))

# testing_generator = DataGenerator(X_test, X_test_fnum, y_test, frame_num, False)
# testPrdct = model.predict_generator(generator=testing_generator, verbose=1, steps=len(X_test) // batch_size,
#                                     max_queue_size=200, workers=32)
# testPrdct = np.squeeze(testPrdct)
# mae = abs(y_test[:len(testPrdct)] - testPrdct).mean()
# print("MAE:", "%.2f" % mae)
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activation_model.summary()
layers = ['time_distributed_13', 'time_distributed_14', 'time_distributed_15', 'time_distributed_16', 'time_distributed_17',
          'time_distributed_18', 'reshape_3', 'conv3d_13', 'conv3d_14', 'conv3d_15', 'max_pooling3d_15']
# layers = ['time_distributed_7', 'time_distributed_8', 'time_distributed_9', 'time_distributed_10', 'time_distributed_11',
#           'time_distributed_12', 'reshape_2', 'conv3d_7', 'conv3d_8', 'conv3d_9', 'max_pooling3d_9']
layers = ['conv3d_1''conv3d_7''time_distributed_3']
output_heatmap(model, 'time_distributed_3', X1)
# plot_layers_map()
# compare_two_layer('time_distributed_12', 'reshape_2', model)
# reshape_4, time_distributed_24
# plot_all_maps('reshape_3')
# compare_all_maps('time_distributed_18', 'reshape_3', model)
